# <span style="color:lightcoral; font-family:freestyle script; font-size:4em">SLOSH till relationsdatabas</span> <br><span style="color:black; font-family:freestyle script; font-size:3em">Frekvenstabeller till SQLite</span>
***

Importera de bibliotek som behövs

In [1]:
import pandas as pd
import numpy as np

Hämta de två första kolumnerna **Variabel** och **Enkät** från *Variabler* ur databasen och kalla för df1.

In [10]:
df1 = pd.DataFrame([['promoted_1', 'slosh 06 work'],
                  ['promoted_2', 'slosh 08 work'],
                  ['workfast_2', 'slosh 08 work'],
                  ['workfast_2', 'slosh 08 non-work'],
                  ['futuwork_4', 'slosh 12 work'],
                  ['futuwork_4', 'slosh 12 non-work']],
                  columns = ['Variabel', 'Enkät'])

In [47]:
df1

,Variabel,Enkät
0,promoted_1,slosh 06 work
1,promoted_2,slosh 08 work
2,workfast_2,slosh 08 work
3,workfast_2,slosh 08 non-work
4,futuwork_4,slosh 12 work
5,futuwork_4,slosh 12 non-work


Läs in SLOSH-datafilen in df2.

In [2]:
df2 = pd.read_excel('testdata frekvenstabell SQL.xlsx')

In [5]:
df2.columns

Index(['promoted_7', 'workfast_7', 'futuwork_7', 'promoted_6', 'workfast_6',
       'futuwork_6', 'promoted_5', 'workfast_5', 'futuwork_5', 'promoted_1',
       'workfast_1', 'futuwork_1', 'promoted_2', 'workfast_2', 'futuwork_2',
       'promoted_3', 'workfast_3', 'futuwork_3', 'promoted_4', 'workfast_4',
       'futuwork_4', 'r_1', 'r_2', 'r_3', 'r_4', 'r_5', 'r_6', 'r_7', 'newid'],
      dtype='object')

För att illustrera hur scriptet funkar behövs bara de variabler som redan finns i relationsdatabasen.

In [6]:
df2 = df2[['promoted_1', 'promoted_2', 'workfast_2', 'futuwork_4', 'r_1', 'r_2', 'r_4']]

In [9]:
df2.head()

,promoted_1,promoted_2,workfast_2,futuwork_4,r_1,r_2,r_4
0,NaN,NaN,NaN,NaN,0,0,0
1,NaN,NaN,NaN,NaN,0,0,0
2,NaN,NaN,NaN,1.0,0,0,2
3,NaN,NaN,NaN,NaN,0,0,0
4,NaN,NaN,NaN,NaN,0,0,0


***  
Exemplet här genomförs för att se ifall värdena i **Enkät** matchas med rätt värden i **r**-kolumnerna i SLOSH-datafilen.  

<span style="color:red">**exempel**

In [61]:
rader = []

for i in df1.index:
    
    variabel = df1.loc[i][0]
    
    enkät = df1.loc[i][1]
    
    if 'non-work' in enkät:
        enkät_id = 1
    else:
            enkät_id = 2    
            
    r = 'r_{}'.format(variabel[-1])
    
    rad = {'Variabel': variabel,
          'Enkät': enkät,
          'E_id': enkät_id,
          'R': r}
    
    rader.append(rad)
    
df4 = pd.DataFrame(rader)

In [62]:
df4

,E_id,Enkät,R,Variabel
0,2,slosh 06 work,r_1,promoted_1
1,2,slosh 08 work,r_2,promoted_2
2,2,slosh 08 work,r_2,workfast_2
3,1,slosh 08 non-work,r_2,workfast_2
4,2,slosh 12 work,r_4,futuwork_4
5,1,slosh 12 non-work,r_4,futuwork_4


<span style="color:red">**exempel slut**  
***

Skapa tabellen *Frekvenstabell*. Exportera till .csv och läs in i SQLite.

In [77]:
rader = [] # varje rad kommer representeras av en dict som läggs i den här listan.
for i in df1.index: # Gå igenom varje rad i df1.
    
    variabel = df1.loc[i][0] # Första elementet i varje sådan rad är ett variabelnamn som här plockas ut.
    
    enkät = df1.loc[i][1] # Gör på samma sätt med enkät (alltså det andra elementet i samma rad).
    
    if 'non-work' in enkät: # Värdena i enkätkolumnen matchas med värden i r-kolumnerna.
        enkät_id = 1
    else:
        enkät_id = 2
        
    r = 'r_{}'.format(variabel[-1]) # För att hitta rätt r-kolumn används sista siffran i variabelnamnet. 
     
    frekvenstabell = df2[df2[r] == enkät_id][variabel].value_counts().sort_index().reset_index() # Ta fram frekvenstabell.
    # För att skilja mellan t.ex. futuwork_4 (w) och futuwork_4 (nw) används r_4. När r_4 = 1 så hör värdet på futuwork_4,
    # på samma rad, till futuwork_4 (w) (och omvänt för futuwork_4 (nw)).
    
    for j in frekvenstabell.index: # Gå igenom varje rad i frekvenstabellen och skapa en dict för varje rad.
        
        rad = {'Variabel': variabel, 
                'Enkät': enkät, 
                'Kod': frekvenstabell.loc[j][0], 
                'Frekvens': frekvenstabell.loc[j][1]}
        
        rader.append(rad) # Lägg varje rad i listan "rader"
        
df3 = pd.DataFrame(rader) # Skapa en dataframe av listan med dicts.

    

In [78]:
df3[['Variabel', 'Enkät', 'Kod', 'Frekvens']]

,Variabel,Enkät,Kod,Frekvens
0,promoted_1,slosh 06 work,1.0,720.0
1,promoted_1,slosh 06 work,2.0,4357.0
2,promoted_2,slosh 08 work,1.0,220.0
3,promoted_2,slosh 08 work,2.0,1639.0
4,promoted_2,slosh 08 work,3.0,7520.0
5,promoted_2,slosh 08 work,4.0,221.0
6,promoted_2,slosh 08 work,5.0,48.0
7,workfast_2,slosh 08 work,1.0,2357.0
8,workfast_2,slosh 08 work,2.0,5716.0
9,workfast_2,slosh 08 work,3.0,1433.0
